# Basit Görüntü Sınıflandırıcı

Bu not defteri, önceden eğitilmiş bir sinir ağı kullanarak görüntüleri nasıl sınıflandıracağınızı gösterir.

**Öğrenecekleriniz:**
- Önceden eğitilmiş bir modeli nasıl yükleyip kullanacağınızı
- Görüntü ön işleme
- Görüntüler üzerinde tahmin yapma
- Güven skorlarını anlama

**Kullanım durumu:** Görüntülerdeki nesneleri tanımlama (örneğin "kedi", "köpek", "araba" gibi)


## Adım 1: Gerekli Kütüphaneleri İçe Aktarma

İhtiyacımız olan araçları içe aktaralım. Henüz hepsini anlamıyorsanız endişelenmeyin!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Adım 2: Önceden Eğitilmiş Modeli Yükleme

**MobileNetV2** adlı, milyonlarca görüntü üzerinde önceden eğitilmiş bir sinir ağını kullanacağız.

Bu yönteme **Transfer Öğrenme** denir - başkası tarafından eğitilmiş bir modeli kullanmak!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Adım 3: Yardımcı Fonksiyonlar

Modelimiz için görüntüleri yüklemek ve hazırlamak üzere fonksiyonlar oluşturalım.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Adım 4: Örnek Görselleri Test Et

İnternetten bazı görselleri sınıflandırmayı deneyelim!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Her Görseli Sınıflandır


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Adım 5: Kendi Görsellerinizi Deneyin!

Aşağıdaki URL'yi sınıflandırmak istediğiniz herhangi bir görsel URL'siyle değiştirin.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Ne Oldu?

1. **Önceden eğitilmiş bir modeli yükledik** - MobileNetV2 milyonlarca görüntü üzerinde eğitildi
2. **Görüntüleri ön işleme tabi tuttuk** - Model için yeniden boyutlandırdık ve formatladık
3. **Model tahminler yaptı** - 1000 nesne sınıfı için olasılık değerleri verdi
4. **Sonuçları çözdük** - Sayıları insan tarafından okunabilir etiketlere dönüştürdük

### Güven Skorlarını Anlamak

- **%90-100**: Çok güvenilir (neredeyse kesin doğru)
- **%70-90**: Güvenilir (muhtemelen doğru)
- **%50-70**: Biraz güvenilir (doğru olabilir)
- **%50'nin altında**: Pek güvenilir değil (belirsiz)

### Tahminler neden yanlış olabilir?

- **Alışılmadık açı veya ışıklandırma** - Model tipik fotoğraflar üzerinde eğitildi
- **Birden fazla nesne** - Model bir ana nesne bekler
- **Nadir nesneler** - Model yalnızca 1000 kategoriyi tanır
- **Düşük kaliteli görüntü** - Bulanık veya pikselli görüntüler daha zordur

---


## 🚀 Sonraki Adımlar

1. **Farklı görseller deneyin:**
   - Görselleri [Unsplash](https://unsplash.com) üzerinden bulun
   - Sağ tıklayın → "Resim adresini kopyala" ile URL alın

2. **Deney yapın:**
   - Soyut sanatla ne oluyor?
   - Farklı açılardan nesneleri tanıyabilir mi?
   - Birden fazla nesneyi nasıl algılıyor?

3. **Daha fazlasını öğrenin:**
   - [Bilgisayarlı Görüntüleme derslerini](../lessons/4-ComputerVision/README.md) keşfedin
   - Kendi görüntü sınıflandırıcınızı eğitmeyi öğrenin
   - CNN'lerin (Evrişimli Sinir Ağları) nasıl çalıştığını anlayın

---

## 🎉 Tebrikler!

Son teknoloji bir sinir ağı kullanarak bir görüntü sınıflandırıcı oluşturdunuz!

Bu teknik şu alanlarda kullanılıyor:
- Google Fotoğraflar (fotoğraflarınızı düzenleme)
- Otonom araçlar (nesneleri tanıma)
- Tıbbi teşhis (röntgenleri analiz etme)
- Kalite kontrol (kusurları tespit etme)

Keşfetmeye ve öğrenmeye devam edin! 🚀



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
